In [39]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
#Importing necessary libraries:
import oursin as urchin
import pandas as pd
urchin.setup()
# from matplotlib.patches import Rectangle

(urchin) Client is already connected. Use ID: 827316a5


In [40]:
# binning raw data
def spikes_bin_data(spike_times_raw_data, spike_clusters_data):
    spike_times_raw_data = np.squeeze(spike_times_raw_data)
    spike_clusters_data = np.squeeze(spike_clusters_data)
    spike_times_sec = spike_times_raw_data / 3e4 # convert from 30khz samples to seconds
    # set up bin edges - 20 ms here
    bins_seconds = np.arange(np.min(spike_times_sec), np.max(spike_times_sec), 0.02)
    # make list of lists for spike times specific to each cluster
    spikes = [spike_times_sec[spike_clusters_data == cluster] for cluster in np.unique(spike_clusters_data)]
    # bin
    binned_spikes = []
    for cluster in spikes:
        counts, _ = np.histogram(cluster, bins_seconds)  
        binned_spikes.append(counts)
    binned_spikes = np.array(binned_spikes) # should be [#neurons, #bins]
    return binned_spikes

In [41]:
# if prev and post aren't exact intervals, flash warning and find nearest bin
def spikes_binned_event_average(binned_spikes, event_start, event_ids, bin_size_sec=0.02, window_start_sec = 0.1, window_end_sec = 0.5):
    bintime_prev = int(window_start_sec * 50)
    bintime_post = int(window_end_sec * 50 + 1)
    windowsize = bintime_prev + bintime_post
    bin_size = bin_size_sec * 1000

    # To bin: divide by 20, floor
    stim_binned = np.floor(event_start * 1000 / bin_size).astype(int)
    stim_binned = np.transpose(stim_binned)


    u_stim_ids = np.unique(event_ids)

    # Initialize final_avg matrix
    final_avg = np.empty((binned_spikes.shape[0], len(u_stim_ids), windowsize))

    for neuron_id in range(binned_spikes.shape[0]):

        for stim_id in u_stim_ids:
            stim_indices = np.where(event_ids[0] == stim_id)[0]

            neuron_stim_data = np.empty((len(stim_indices), windowsize))
            
            for i, stim_idx in enumerate(stim_indices):
                bin_id = int(stim_binned[0][stim_idx])
                selected_columns = binned_spikes[neuron_id, bin_id - bintime_prev: bin_id + bintime_post]
                neuron_stim_data[i,:] = selected_columns

            bin_average = np.mean(neuron_stim_data, axis=0)/bin_size_sec
            final_avg[neuron_id, int(stim_id) - 1, :] = bin_average
    return final_avg


In [42]:
# processing data appropriately
from pathlib import Path
dpath = Path(r'../../data/')
st_samp = np.load(dpath / 'spike_times.npy')
sc = np.load(dpath / 'spike_clusters.npy')
binned_spikes = spikes_bin_data(st_samp,sc)
event_start = np.load("../../data/natImsOnTimes.npy")
event_ids = np.load("../../data/natImsIDs.npy") 
prepped_data = spikes_binned_event_average(binned_spikes,event_start,event_ids)

In [80]:
# pulling in location data
locations = pd.read_csv("../../data/allen_visual_behavior_firing_rate_modified.csv")
locations = locations.iloc[::400]
locations = locations.iloc[:753]
locations.reset_index(drop=True, inplace = True)
locations

,unit_id,left_right_ccf_coordinate,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,firing_rate,percentile_rank,color,size_scale
0,1049373121,7369,8149,3916,3.087015,0.494522,#9f7fbe,0.059229
1,1049374828,7522,7948,1446,0.303878,0.166482,#ddd2e8,0.042162
2,1049375242,7635,8838,3097,0.865969,0.254040,#ccbcdd,0.046166
3,1049375653,8807,8436,3000,23.292304,0.959434,#470b83,0.095884
4,1049376065,8010,8653,3520,4.770111,0.622959,#875fae,0.067660
...,...,...,...,...,...,...,...,...
748,1179687886,7785,8102,3279,23.573880,0.960422,#470a83,0.095982
749,1179688295,6774,8399,3813,17.889792,0.931918,#4c1286,0.093189
750,1179688724,8609,7763,1112,7.807732,0.763063,#6c3c9c,0.078231
751,1179689171,8946,8042,1383,0.006436,0.018244,#f9f7fb,0.036159


In [64]:
# def plot_event_average_interaction(): #wrapper function
    
#graph: slider for time, dropdown for which id to show

In [65]:
def slope_viz_stimuli_per_neuron(t=-100, neuron_id = 0):
    
    # Plotting data:
    for i in range(0,prepped_data.shape[1]):
        y = prepped_data[neuron_id][i]
        x = np.arange(-100, 520, step=20)
        plt.plot(x,y)

    # Labels:
    plt.xlabel('Time from stimulus onset')
    plt.ylabel('Number of Spikes Per Second')
    plt.title(f'Neuron {neuron_id} Spiking Activity with Respect to Each Stimulus')

    #Accessories:
    plt.axvspan(0, 300, color='gray', alpha=0.3)
    plt.axvline(t, color='red', linestyle='--',)
    # Set y-axis limits
     # Calculate y-axis limits
    max_y = max([max(prepped_data[neuron_id][i]) for i in range(10)])  # Maximum y-value across all lines
    if max_y < 10:
        max_y = 10  # Set ymax to 10 if the default max is lower than 10
    plt.ylim(0, max_y)
   
    # plt.legend()
    plt.show()

In [66]:
#so new version would take just the neurons that are being changed in size, and the positions that are being changed
# [neuron][stim] stayes the same it's just time 3rd col time that's being manipulated as slider moves
def update_neuron_sizing(stim_id, t=-100):
    global neurons
    
    t = round((t+100)/20)
        
    size_list = []
    for i in range(binned_spikes.shape[0]):
        size = round(prepped_data[i][stim_id][t]/100,1)
        # size = (round(row.percentile_rank,1))
        size_list.append(size)


    urchin.particles.set_sizes(neurons, size_list)

In [67]:
def slope_viz_neurons_per_stimuli(t = -100, stim_id = 0):
    # Plotting data:
    for i in range(0,prepped_data.shape[0]):
        y = prepped_data[i][stim_id]
        x = np.arange(-100, 520, step=20)
        plt.plot(x,y)
    
    # Labels:
    plt.xlabel(f'Time from Stimulus {stim_id} display (20 ms bins)')
    plt.ylabel('Number of Spikes Per Second')
    plt.title(f'Neuron Spiking Activity with Respect to Stimulus ID {stim_id}')

    # Accessories:
    plt.axvspan(0, 300, color='gray', alpha=0.3)
    plt.axvline(t, color='red', linestyle='--',)

    plt.show()

    update_neuron_sizing(stim_id, t)
    

In [68]:
# time_slider = widgets.IntSlider(value=-100, min=-100, max=500, step=5, description='Time')
# time_slider.layout.width = '6.55in'
# time_slider.layout.margin = '0 -15px'

# neuron_dropdown = widgets.Dropdown(
#     options= range(0,prepped_data.shape[0]),
#     value=355,
#     description='Neuron ID:',
# )
# neuron_dropdown.layout.margin = "20px 20px"

# stimuli_dropdown = widgets.Dropdown(
#     options= range(0,prepped_data.shape[1]),
#     value=0,
#     description='Stimulus ID:',
# )
# stimuli_dropdown.layout.margin = "20px 20px"

# select_view_button = widgets.RadioButtons(
#     options=[("Stimulus View", "stim"), ("Neuron View", 'neuron')],
#     value = 'stim',
#     description='Select View:',
# )

In [69]:
# # Display interactive neuron graph:
# output = widgets.interactive_output(slope_viz_stimuli_per_neuron(prepped_data), {'t': time_slider, 'neuron_id': neuron_dropdown})

# # Display the widgets and the output
# display(widgets.VBox([neuron_dropdown,time_slider]))
# display(output)

In [70]:
# # Display interactive stimulus graph:
# output = widgets.interactive_output(slope_viz_neurons_per_stimuli(prepped_data), {'t': time_slider, 'stim_id': stimuli_dropdown})
# # Display the widgets and the output
# display(widgets.VBox([stimuli_dropdown,time_slider]))
# display(output)

In [71]:
def plot_appropriate_graph(view="stim"):
    time_slider = widgets.IntSlider(value=-100, min=-100, max=500, step=5, description='Time')
    time_slider.layout.width = '6.53in'
    time_slider.layout.margin = '0 -4px'
    
    if view == "stim":
        stimuli_dropdown = widgets.Dropdown(
            options= range(0,prepped_data.shape[1]),
            value=0,
            description='Stimulus ID:',
        )
        stimuli_dropdown.layout.margin = "20px 20px"
        output = widgets.interactive_output(slope_viz_neurons_per_stimuli, {'t': time_slider, 'stim_id': stimuli_dropdown})
        # Display the widgets and the output
        display(widgets.VBox([stimuli_dropdown,time_slider]))
        display(output)
    
    elif view == "neuron":
        neuron_dropdown = widgets.Dropdown(
            options= range(0,prepped_data.shape[0]),
            value=355,
            description='Neuron ID:',
        )
        neuron_dropdown.layout.margin = "20px 20px"

        # Link the function with the interact function
        output = widgets.interactive_output(slope_viz_stimuli_per_neuron, {'t': time_slider, 'neuron_id': neuron_dropdown})

        # Display the widgets and the output
        display(widgets.VBox([neuron_dropdown,time_slider]))
        display(output)


Incorporating Urchin portion below:

In [72]:
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [75]:
urchin.ccf25.grey.set_visibility(True)
urchin.ccf25.grey.set_material('transparent-unlit')
urchin.ccf25.grey.set_color('#000000')
urchin.ccf25.grey.set_alpha(0.1)

In [81]:
#Setting positions (static):
positions_list = []

for i, row in locations.iterrows():
    position = [round(row.left_right_ccf_coordinate), round(row.anterior_posterior_ccf_coordinate), round(row.dorsal_ventral_ccf_coordinate)]
    positions_list.append(position)    

In [82]:
urchin.particles.clear()

In [83]:
urchin.particles.set_material('circle')
neurons = urchin.particles.create(len(locations))
urchin.particles.set_positions(neurons, positions_list) #setting the positions within the renderer

In [85]:
plot_appropriate_graph("stim")

Output()

In [58]:
## figure out at what point this gets called in the previous function, and how to feed in the stim data to the function

In [59]:
# create initial size list, and save existing order

# when slide is changed:
    # if new size is different from initial
        # create new sizing array to feed into urchin
        # update size within initial size list

